# K-Nearest Neighbours Classifier

<span>KNN is a very simple machine learning algorithim. Given a distance parameter and nearest neighbours parameter. The algorithim use premise that points that are related to one class are closely associated with each other. The algorithim will group each random data points place hyperplane using its K-Nearest Neighbours.</span>

### Import Dataset

In [31]:
%matplotlib inline
%pylab inline
 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn
import warnings

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
 
pd.set_option('max_columns',1000)
pd.set_option('max_rows',30)
 
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pylab.rcParams['figure.figsize'] = (8.0, 7.0)
warnings.filterwarnings('ignore');

Populating the interactive namespace from numpy and matplotlib


### Import Data

In [32]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target


train_x, test_x, train_y, text_y = train_test_split(X,y)

### Define the Model

In [60]:
model = KNeighborsClassifier(n_neighbors=5)

### Fit the Model

In [61]:
model.fit(train_x, train_y);

### Cross Validation Score

In [34]:
scores = cross_val_score(model, train_x, train_y, cv=10,
               scoring='accuracy')
print(f'Cross Validation Score: {scores.mean():.5f}')

Cross Validation Score: 0.93892


### Confustion Matrix

In [59]:
# Training Confusion Matrix
from sklearn.metrics import confusion_matrix
cmatrix = pd.DataFrame(confusion_matrix(train_y, model.predict(train_x)))
cmatrix.index.name = 'class'

cmatrix['result'] = 'actual'
cmatrix.set_index('result', append=True, inplace=True)
cmatrix = cmatrix.reorder_levels(['result', 'class'])
cmatrix = cmatrix.stack()
cmatrix = pd.DataFrame(cmatrix)
cmatrix.columns = ['prediction']
cmatrix.unstack()

prediction     
                      0    1
result class                
actual 0            138   13
       1             10  265

### Predict Results

In [66]:
pd.Series(model.predict(test_x)).head(n=7)

0    1
1    1
2    0
3    1
4    0
5    0
6    1
dtype: int64

Author: Kavi Sekhon